# Sat Chirp Backgrounder
## By Terry Bondy, VA3TYB

In [94]:
printf(strftime ("Last updated: %A %e %B %Y", localtime (time ())))

Last updated: Saturday 26 October 2019

# Scope
What is described here is some background for a means of determining the 
instantaneous translating properties of a linear satellite by sending 
a single brief 'chirp' in the uplink and listening across the downlink satellite band 
with an SDR and doing some processing on the received signal. These translation 
properties include:
* The instantaneous translation frequency,
* Whether the received downlink signal is received USB or LSB.

# Producing This Document
This document is produced using [Octave](https://www.gnu.org/software/octave/), an open source tool very similar to
*Matlab*. The lines in the boxes are *Octave* commands used to create the 
mathematical examples that are used. Octave is being run in a [Jupyter](https://jupyterlab.readthedocs.io/en/stable/) notebook which can be accessed and run from the internet at [this link](https://mybinder.org/v2/gh/tbondy760/sat-chirp-notebooks/master?filepath=SatChirpBackgrounder.ipynb).

But first a bit of bookkeeping to make the plots look nice.

In [ ]:
%plot --format svg

# Matched Filter - Time Domain
As described in a previous section, an SDR will receive the complete satellite downlink
band and do processing on it while a 'chirp' is sent on the uplink. It will look at the 
_frequency_ representation of the received chirp and determine its frequency offset from the satellite downlink
band edge.

In order to determine the frequency offset, the process will use a 
[matched filter](https://en.wikipedia.org/wiki/Matched_filter). Some examples
of a matched filter will demonstrate the two properties that will make it 
useful for this process:
* It is easy to determine whether the expected signal used to create the matched filter is present,
* It is easy to determine _when_ the expected used to create the matched filter has completed.

For this example the expected signal to detect will be a 20 Hz cosine pulse 1s long (Figure 1).

In [ ]:
sample_rate = 6000; # Sampling at 6 kHz
T_base = [0: 1/sample_rate: 1]; # The time base, sampling at 6 kHz
V_expected = cos(2*pi*20*T_base);

plot(T_base, V_expected)
ylabel("Amplitude (V)")
xlabel("Time (s)")
title("Figure 1: Cosine Pulse - Time Domain")
grid on
grid minor
axis([-0.5 1.5 -2 2], "tic")

In the time domain, a matched filter is created by time reversing the signal to match or expected signal:
$$
f_{matching}(t) = f_{expected}(-t)\;\;\;...(1)
$$
In this case the matched filter is the signal itself because $cos(2\pi20t) = cos(-2\pi20t)$.

The output of the filter is determined by convolving the input signal with the matched filter.

For the first part of the example, the received signal is the expected signal itself followed by 4 sec of silence, three times (Figure 2).

In [ ]:
V_4s_silence = zeros(1, sample_rate*4);
V_received = horzcat(V_expected, V_4s_silence, V_expected, V_4s_silence, V_expected, V_4s_silence);
T_base = [0: 1/sample_rate: (size(V_received,2)-1)/sample_rate]; # The time base

plot(T_base, V_received)
ylabel("Amplitude (V)")
xlabel("Time (s)")
title("Figure 2: Received Signal - Time Domain")
grid on
grid minor
axis([-0.5 15.5 -2 2], "tic")

To do the matching the matching filter, the expected signal time reversed, is convolved with the received signal. The maximal peaks of the output indicates (Figure 3):
* Where the expected signal was detected,
* When the expected signal was completely received.

In [ ]:
V_matching = flip(V_expected); # 'flip' does the time reversing, not really necessary for this signal
V_matched = conv(V_matching, V_received);
T_base = [0: 1/sample_rate: (size(V_matched,2)-1)/sample_rate]; # The time base, sampling at 6 kHz

plot(T_base, V_matched)
ylabel("Energy (V^2)")
xlabel("Time (s)")
title("Figure 3: Matched Filter Output - Time Domain")
grid on
grid minor
axis([-0.5 15.5], "tic")

For the next part of the example, it will be shown that the expected signal
can be detected in the presence of a large amount of noise (i.e. SNR = -20 dB).

In [ ]:
# Begin credited work
# The following work derives from work done and presented by Mathuranathan Viswanathan at 
# https://www.gaussianwaves.com/2015/06/how-to-generate-awgn-noise-in-matlaboctave-without-using-in-built-awgn-function/
snr = -20; # dB
# dB is logarithmic, linearize the SNR
snr_lin = 10^(snr/10);
# Determine the signal power
e_sig = sum(abs(V_expected).^2)/size(V_expected,2);
# And then get the noise spectral density
n_0 = e_sig/snr_lin;
# For real valued vectors
n_sigma = sqrt(n_0);
V_noise = n_sigma * randn(1,size(V_received,2));
# End credited work

V_noisy_received = V_received .+ V_noise;
T_base = [0: 1/sample_rate: (size(V_received,2)-1)/sample_rate]; # The time base

plot(T_base, V_noisy_received)
ylabel("Amplitude (V)")
xlabel("Time (s)")
title("Figure 4: Received Signal with Noise - Time Domain")
grid on
grid minor
axis([-0.5 15.5], "tic")

Can the pulses bee seen amongst the noise? A plot is shown with the original received signal superimposed for magnitude comparision (Figure 5).

In [ ]:
plot(T_base, V_noisy_received, T_base, V_received)
ylabel("Amplitude (V)")
xlabel("Time (s)")
title("Figure 5: Magnitude Comparision of Cosine Pulses and Noise - Time Domain")
grid on
grid minor
axis([-0.5 15.5], "tic")

Examine the matched filter output (Figure 6).

In [ ]:
V_matched = conv(V_matching, V_noisy_received);
T_base = [0: 1/sample_rate: (size(V_matched,2)-1)/sample_rate]; # The time base, sampling at 6 kHz
plot(T_base, V_matched)
ylabel("Energy (V^2)")
xlabel("Time (s)")
title("Figure 6: Matched Filter Output of Noisy Received Signal - Time Domain")
grid on
grid minor
axis([-0.5 15.5], "tic")

So even though the cosine pulses are 20 dB less than the noise, the matched filter
is still able to detect them.

# A Brief Review of Signal Analysis In The Time and Frequency Domains 
Before taking on using a matched filter in the _frequency_ domain, a short review of signal analysis in both the time and frequency domains is presented.

Consider again a 20 Hz cosine pulse lasting 1s (Figure 7).

In [ ]:
T_base = [0: 1/sample_rate: 1]; # The time base, sampling at 6 kHz
V_cos_pulse = cos(2*pi*20*T_base);

plot(T_base, V_cos_pulse)
ylabel("Amplitude (V)")
xlabel("Time (s)")
title("Figure 7: Cosine Pulse - Time Domain")
grid on
grid minor
axis([-0.5 1.5 -2.5 2.5], "tic")

A fast-fourier-transform (FFT) is used to determine the frequency components of the signal (Figure 8).

The Octave function `fft` is used to do the FFT. The Octave function `fftshift` is used to shift 0 Hz to the 
centre of the resulting vector of results.

In [ ]:
V_of_f = fftshift(fft(V_cos_pulse));
F_base = [-sample_rate/2: 1: sample_rate/2];
plot(F_base, V_of_f)
ylabel("Relative Energy (V^2)")
xlabel("Frequency (Hz)")
title("Figure 8: Cosine Pulse - Frequency Domain")
grid on
grid minor
axis([-25 25], "tic")

The energy at +/- 20 Hz can be seen, although the resolution is lacking. To increase the resolution, the time sample can be extended. If the frequency resolution was only 1 Hz when the sample was only 1s long, it will be 0.1 Hz when the sample is 10s long.

In [ ]:
V_9s_silence = zeros(1, sample_rate*9);
V_of_f = fftshift(fft(horzcat(V_cos_pulse, V_9s_silence)));
F_base = [-sample_rate/2: 1/10: sample_rate/2];
plot(F_base, V_of_f)
ylabel("Relative Energy (V^2)")
xlabel("Frequency (Hz)")
title("Figure 9: Cosine Pulse - Frequency Domain")
grid on
grid minor
axis([-35 35], "tic")

It can be seen from the graph:
* That the primary components consist of frequencies at and around +20 Hz and -20 Hz,
* There is "splatter" approx 15 Hz on either side of the primary +/- 20 Hz components,
* Real value signals will have +ve and -ve frequency components.

The appearance of splatter can be reasoned by considering the cosine pulse as being modulated by an square pulse 1s long. To reduce the splatter, we "soften" our cosine pulse by modulating instead with a [raised cosine pulse](https://en.wikipedia.org/wiki/Raised-cosine_filter).

In [ ]:
T_base = [0: 1/sample_rate: 1]; # The time base, sampling at 6 kHz
V_cos_pulse = cos(2*pi*20*T_base) .* (cos(2*pi*(T_base - 0.5)) + 1)/2;

plot(T_base, V_cos_pulse)
ylabel("Amplitude (V)")
xlabel("Time (s)")
title("Figure 10: Cosine Pulse with Raised Cosine Modulation - Time Domain")
grid on
grid minor
axis([-0.5 1.5 -2.5 2.5], "tic")

So let's examine this in the frequency domain.

In [ ]:
V_9s_silence = zeros(1, sample_rate*9);
V_of_f = fftshift(fft(horzcat(V_cos_pulse, V_9s_silence)));
F_base = [-sample_rate/2: 1/10: sample_rate/2];
plot(F_base, V_of_f)
ylabel("Relative Energy (V^2)")
xlabel("Frequency (Hz)")
title("Figure 11: Cosine Pulse with Raised Cosine Modulation - Frequency Domain")
grid on
grid minor
axis([-35 35], "tic")

The same process can be used
with complex phasors by using `exp(i*2*pi*20*T)` or `exp(-i*2*pi*20*T)` in 
place of `cos(2*pi*20*T)`. For the +ve phasor the time signal looks the 
same (Figure 12).

In [ ]:
T_base = [0: 1/sample_rate: 1]; # The time base, sampling at 6 kHz
V_cos_pulse = exp(i*2*pi*20*T_base) .* (cos(2*pi*(T_base - 0.5)) + 1)/2;

plot(T_base, V_cos_pulse)
ylabel("Amplitude (V)")
xlabel("Time (s)")
title("Figure 13: exp(i*2*pi*20*t) Pulse with Raised Cosine Modulation - Time Domain")
grid on
grid minor
axis([-0.5 1.5 -2.5 2.5], "tic")

And in the frequency domain (Figure 14).

In [ ]:
V_9s_silence = zeros(1, sample_rate*9);
V_of_f = fftshift(fft(horzcat(V_cos_pulse, V_9s_silence)));
F_base = [-sample_rate/2: 1/10: sample_rate/2];
plot(F_base, V_of_f)
ylabel("Relative Energy (V^2)")
xlabel("Frequency (Hz)")
title("Figure 14: exp(i*2*pi*20*t) Pulse with Raised Cosine Modulation - Frequency Domain")
grid on
grid minor
axis([-35 35], "tic")

Looking at the figure it is very similar to Figure 11 except:
* That there is only one sideband, the USB,
* That the sideband has twice the energy.

Now for `exp(-i*2*pi*20*T)` (Figure 15). 

In [ ]:
T_base = [0: 1/sample_rate: 1]; # The time base, sampling at 6 kHz
V_cos_pulse = exp(-i*2*pi*20*T_base) .* (cos(2*pi*(T_base - 0.5)) + 1)/2;

plot(T_base, V_cos_pulse)
ylabel("Amplitude (V)")
xlabel("Time (s)")
title("Figure 16: exp(-i*2*pi*20*t) Pulse with Raised Cosine Modulation - Time Domain")
grid on
grid minor
axis([-0.5 1.5 -2.5 2.5], "tic")

It looks identical to Figure 13. And in the frequency domain (Figure 17).

In [ ]:
V_9s_silence = zeros(1, sample_rate*9);
V_of_f = fftshift(fft(horzcat(V_cos_pulse, V_9s_silence)));
F_base = [-sample_rate/2: 1/10: sample_rate/2];
plot(F_base, V_of_f)
ylabel("Relative Energy (V^2)")
xlabel("Frequency (Hz)")
title("Figure 17: exp(i*2*pi*20*t) Pulse with Raised Cosine Modulation - Frequency Domain")
grid on
grid minor
axis([-35 35], "tic")

Looking at the figure it is very similar to Figure 11 except:
* That there is only one sideband, the LSB,
* That the sideband has twice the energy.

We have also indirectly demonstrated the trig identity
$$
cos(x) = \frac{e^{ix} + e^{-ix}}{2}
$$

# Matched Filter In The Frequency Domain
So far in the examples a matched filter is shown working in the time domain.
Matched filters can also be used in the frequency domain. In this case it
is possible to determine the frequency offset of the signal. The process
is the following:
* RF pulse $	\rightarrow$ FFT $	\rightarrow$ flip in frequency = matched filter
* input signal $	\rightarrow$ FFT $	\rightarrow$ convolve with matched filter = matched filter output

There is a property in time/frequency analysis that can be brought to bear,
specifically convolution of two signals in one domain is multiplication in
the other. So now the process becomes:
* RF pulse $	\rightarrow$ FFT $	\rightarrow$ flip in frequency $	\rightarrow$ reverse FFT $	\rightarrow$ matched filter in time domain
* input signal $	\rightarrow$ multiply by matched filter in time domain $	\rightarrow$ FFT = matched filter output

For this example we are going to use the pulse from Figure 13, namely

`V_cos_pulse = exp(i*2*pi*20*T_base) .* (cos(2*pi*(T_base - 0.5)) + 1)/2`

And when it occurs, we want to read a 0 offset from the frequency graph. In order for it to do this, the input signal should be proportional to:
$$\frac{cos(2\pi (t-0.5))+1}{2}, -0.5 <= t <= 0.5$$

Let's look at that function and its frequency response (Figure 18 & Figure 19)

In [ ]:
T_base = [0: 1/sample_rate: 1]; # The time base, sampling at 6 kHz
V_cos_pulse = (cos(2*pi*(T_base - 0.5)) + 1)/2;

plot(T_base, V_cos_pulse)
ylabel("Amplitude (V)")
xlabel("Time (s)")
title("Figure 18: DC Raised Cosine Pulse - Time Domain")
grid on
grid minor
axis([-0.5 1.5 -2.5 2.5], "tic")

In [ ]:
V_9s_silence = zeros(1, sample_rate*9);
V_of_f = fftshift(fft(horzcat(V_cos_pulse, V_9s_silence)));
F_base = [-sample_rate/2: 1/10: sample_rate/2];
plot(F_base, V_of_f)
ylabel("Relative Energy (V^2)")
xlabel("Frequency (Hz)")
title("Figure 19: DC Raised Cosine Pulse - Frequency Domain")
grid on
grid minor
axis([-35 35], "tic")

But the DC raised cosine pulse is what we want to end up with _after_ the multiplication of the expected input signal with the sampling function, or:
$$
V_{matched}(t) = \frac{cos(2\pi (t-0.5))+1}{2} = V_{expected input}(t) V_{sampling}(t)
$$
If
$$
V_{sampling}(t) = \frac{cos(2\pi (t-0.5))+1}{2} V'_{sampling}(t)
$$
then
$$
1 = V_{expected input}(t) V'_{sampling}(t) 
$$
or
$$
V'_{sampling}(t) = \frac{1}{V_{expected input}(t)}
$$

If we are going to use the pulse from Figure 13 as the expected input, or

`V_cos_pulse = exp(i*2*pi*20*T_base) .* (cos(2*pi*(T_base - 0.5)) + 1)/2`

then

`V_matching = 1/(exp(i*2*pi*20*T_base)) .* (cos(2*pi*(T_base - 0.5)) + 1)/2`

So let's try this with a signal at 120 Hz, or the same as our expected input signal but shifted up 100 Hz, and see what the time and frequency graphs indicate (Figure 20 and Figure 21 respectively).

In [ ]:
V_matching = 1 ./ (exp(i*2*pi*20*T_base)) .* (cos(2*pi*(T_base - 0.5)) + 1)/2;
V_new_signal = exp(i*2*pi*120*T_base); # Notice the 120 term
V_matched = V_matching .* V_new_signal;

plot(T_base, V_matched)
ylabel("Amplitude (V)")
xlabel("Time (s)")
title("Figure 20: Matched Signal When Expected Signal 100 Hz Up - Time Domain")
grid on
grid minor
axis([-0.5 1.5 -2.5 2.5], "tic")

In [ ]:
V_9s_silence = zeros(1, sample_rate*9);
V_of_f = fftshift(fft(horzcat(V_matched, V_9s_silence)));
F_base = [-sample_rate/2: 1/10: sample_rate/2];
plot(F_base, V_of_f)
ylabel("Relative Energy (V^2)")
xlabel("Frequency (Hz)")
title("Figure 21: Matched Signal When Expected Signal 100 Hz Up - Frequency Domain")
grid on
grid minor
axis([-15 115], "tic")

Let's see the performance in the face of noise using an SNR of -20 dB again.

In [ ]:
snr = -20; # dB
# Linearize the SNR
snr_lin = 10^(snr/10);
# Determine the original signal power
e_sig = sum(abs(V_new_signal).^2)/size(V_new_signal,2);
# And then get noise spectral density
n_0 = e_sig/snr_lin;
n_sigma = sqrt(n_0);

# This time the noise has to have both real and complex components because the input signal does
V_noise = n_sigma * (randn(1,size(V_new_signal,2)) .+ i * randn(1,size(V_new_signal,2)));

V_matched = V_matching .* (V_new_signal .+ V_noise);

plot(T_base, V_matched)
ylabel("Amplitude (V)")
xlabel("Time (s)")
title("Figure 22: Noisy Matched Signal When Expected Signal 100 Hz Up - Time Domain")
grid on
grid minor
axis([-0.5 1.5 ], "tic")

In [ ]:
V_9s_silence = zeros(1, sample_rate*9);
V_of_f = fftshift(fft(horzcat(V_matched, V_9s_silence)));
F_base = [-sample_rate/2: 1/10: sample_rate/2];
plot(F_base, V_of_f)
ylabel("Relative Energy (V^2)")
xlabel("Frequency (Hz)")
title("Figure 23: Noisy Matched Signal When Expected Signal 100 Hz Up - Frequency Domain")
grid on
grid minor
axis([-15 115], "tic")

It can be seen that the offset of the expected signal can read off the frequency graph even when the expected signal is swamped with noise.

# Summary
In this lab book I have:
* Introduced the matched filter, how it works in the time domain, and shown how well it works in the face of noise,
* Refreshed how frequency domain analysis of a time domain signal can be done using the fast fourier transform (FFT),
* Shown how the matched filter in the _frequency_ domain can be used to determine the frequency offset of an expected signal.

In an upcoming lab book I'll determine what makes a good expected signal to be used for the matching filter and how it can be generated.